In [2]:
!pip install pycaret

  Using cached pycaret-2.3.1-py3-none-any.whl (261 kB)
  Using cached wordcloud-1.8.1-cp38-cp38-manylinux1_x86_64.whl (371 kB)
  Using cached textblob-0.15.3-py2.py3-none-any.whl (636 kB)
  Using cached scikit_learn-0.23.2-cp38-cp38-manylinux1_x86_64.whl (6.8 MB)
  Using cached pyLDAvis-3.3.1-py2.py3-none-any.whl
  Using cached Boruta-0.3-py3-none-any.whl (56 kB)
  Using cached pyod-0.8.8-py3-none-any.whl
  Using cached kmodes-0.11.0-py2.py3-none-any.whl (18 kB)
  Using cached mlflow-1.17.0-py3-none-any.whl (14.2 MB)
  Using cached numpy-1.19.5-cp38-cp38-manylinux2010_x86_64.whl (14.9 MB)
  Using cached yellowbrick-1.3.post1-py3-none-any.whl (271 kB)
  Using cached umap_learn-0.5.1-py3-none-any.whl
  Using cached pandas_profiling-3.0.0-py2.py3-none-any.whl (248 kB)
  Using cached mlxtend-0.18.0-py2.py3-none-any.whl (1.3 MB)
  Using cached lightgbm-3.2.1-py3-none-manylinux1_x86_64.whl (2.0 MB)
  Using cached cufflinks-0.17.3-py3-none-any.whl
  Using cached imbalanced_learn-0.7.0-py3-non

  Using cached multimethod-1.4-py2.py3-none-any.whl (7.3 kB)
  Using cached ImageHash-4.2.0-py2.py3-none-any.whl (295 kB)
     |████████████████████████████████| 25.8 MB 2.2 MB/s eta 0:00:01
  Using cached retrying-1.3.3-py3-none-any.whl
  Using cached blis-0.7.4-cp38-cp38-manylinux2014_x86_64.whl (9.8 MB)
  Using cached murmurhash-1.0.5-cp38-cp38-manylinux2014_x86_64.whl (20 kB)
  Using cached preshed-3.0.5-cp38-cp38-manylinux2014_x86_64.whl (130 kB)
  Using cached srsly-1.0.5-cp38-cp38-manylinux2014_x86_64.whl (186 kB)
  Using cached cymem-2.0.5-cp38-cp38-manylinux2014_x86_64.whl (35 kB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached plac-1.1.3-py2.py3-none-any.whl (20 kB)
  Using cached wasabi-0.8.2-py3-none-any.whl (23 kB)
  Using cached thinc-7.4.5-cp38-cp38-manylinux2014_x86_64.whl (1.1 MB)
  Using cached alembic-1.4.1-py2.py3-none-any.whl
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached docker-5.0.0-py2.py3-none-an

In [3]:
import argparse
import numpy as np
import pandas as pd
import ast
from pycaret import *

In [1]:
def text_to_dict(df):
    dict_columns = ['belongs_to_collection', 'genres', 'spoken_languages', 'production_companies',
                    'production_countries', 'Keywords', 'cast', 'crew']
    for columns in dict_columns:
        df[columns] = df[columns].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x))
    return df

def preprocess(df):
    dict_columns = ['belongs_to_collection', 'genres', 'spoken_languages', 'production_companies',
                    'production_countries', 'Keywords', 'cast', 'crew']

    df = text_to_dict(df)
    df.release_date = pd.to_datetime(df.release_date)
    df['release_year'] = df.release_date.apply(lambda x: x.year)
    df['release_month'] = df.release_date.apply(lambda x: x.month)
    df['director'] = df.crew.apply(lambda x: [y['name'] for y in x if y['job'] == 'Director'])
    df['director'] = df['director'].apply(lambda x: x[0] if len(x) > 0 else None)
    df["budget"] = df["budget"].apply(lambda x: x if x != 0 else None)
    genres = {'Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','History'
        ,'Horror','Music','Mystery','Romance','Science Fiction','TV Movie','Thriller','War','Western'}
    for genre in genres:
        df[genre] = 0
    for index, row in df.iterrows():
        row_genres = row["genres"]
        row_genres = [x["name"] for x in row_genres]
        for genre in genres:
            if genre in row_genres:
                df.at[index, genre] = 1
    df.drop(["genres"], axis='columns', inplace=True)
    cols_to_drop = ['backdrop_path', 'poster_path', 'homepage', 'popularity', 'imdb_id', 'video', 'status'
        , 'original_title', 'overview', 'production_companies', 'production_countries', 'spoken_languages',
                    'title', 'tagline', 'Keywords', 'cast', 'crew']
    df.drop(cols_to_drop, axis='columns', inplace=True)


    return df

In [5]:
train_data = preprocess(pd.read_csv("train.tsv", sep="\t"))
test_data = preprocess(pd.read_csv("test.tsv", sep="\t"))

In [5]:
pycar = setup(data=train_data, test_data=test_data, target='revenue',)

,Description,Value
0,session_id,6468
1,Target,revenue
2,Original Data,"(5215, 30)"
3,Missing Values,True
4,Numeric Features,5
5,Categorical Features,23
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(5215, 3372)"


In [6]:
top3 = compare_models(n_select = 3)
print(top3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,29103785.9281,4125631795408029.5000,63450141.9869,0.7902,2.5124,17712.8630,0.2540
rf,Random Forest Regressor,28683905.0580,4560845311887492.0000,66520044.8785,0.7730,2.2999,12421.0607,13.6050
et,Extra Trees Regressor,29085808.6639,4629244939421503.0000,67127415.4650,0.7714,2.2390,10069.5311,25.1300
gbr,Gradient Boosting Regressor,32419101.6324,4653245940349745.0000,67377404.1738,0.7658,2.7943,27572.3845,4.9220
ridge,Ridge Regression,41055939.6000,5390658657740390.0000,72737720.4000,0.7279,3.0911,67578.9858,0.3980
llar,Lasso Least Angle Regression,38707742.1668,5661332348953735.0000,74574134.7624,0.7132,3.0759,59501.0599,0.7600
en,Elastic Net,41029427.2000,5691858808628838.0000,74874536.0000,0.7105,3.1579,71636.9272,7.6590
lr,Linear Regression,42118950.0000,5880088944757965.0000,76135064.8000,0.7004,3.1825,71880.2372,11.4420
omp,Orthogonal Matching Pursuit,40304508.7901,7051740350689754.0000,83083218.0672,0.6476,2.9816,50126.5649,0.5670
lasso,Lasso Regression,47606966.4000,7299539102020403.0000,84783821.6000,0.6311,3.1485,71600.5837,6.6290


[LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.1, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
              random_state=6468, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0), RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=6468, verbose=0, warm_start=False), ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.

In [7]:
tuned_models = [tune_model(m) for m in top3]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,30337012.3364,4422992520186097.0000,66505582.6242,0.8183,2.1391,4938.2216
1,28112779.1872,4408985989869383.0000,66400195.7066,0.7316,2.5243,10473.7285
2,26770701.9956,3127297646303049.0000,55922246.4347,0.8103,2.5829,20742.5734
3,34089485.4100,6105204990553132.0000,78135811.1915,0.8142,2.6320,20780.2411
4,30918473.2771,4657890117174237.0000,68248737.1105,0.8019,2.3348,9791.0642
5,33106259.9776,6597600770447728.0000,81225616.4670,0.7438,2.6275,18460.5834
6,29159417.9752,3879593718463856.0000,62286384.6957,0.7830,2.6127,12417.4209
7,31303089.1305,3535698522860145.0000,59461739.9919,0.6924,2.7697,13551.8918
8,28466037.1435,2890436381779827.0000,53762778.7766,0.8411,2.7715,75038.3652
9,27957174.7180,2536937570478373.0000,50368021.3080,0.8574,2.6168,44988.4434


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:18:44
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,MAE,MSE,RMSE,R2,RMSLE,MAPE


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


WorkerInterrupt: 